# Set the stage

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import cvxpy

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Load an image

In [ ]:
from skimage import io
from PIL import Image 
from torchvision import transforms

# read the image, resize to 224 and convert to PyTorch Tensor
pig_img = Image.open("pig.jpg")
preprocess = transforms.Compose([
   transforms.Resize(224),
   transforms.ToTensor(),
])
pig_tensor = preprocess(pig_img)[None,:,:,:]

# plot image (note that numpy using HWC whereas Pytorch user CHW, so we need to convert)
plt.imshow(pig_tensor[0].numpy().transpose(1,2,0))

# Load a model

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

mnist_train = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("./data", train=False, download=True, transform=transforms.ToTensor())

train_idx = (mnist_train.train_labels <= 1)
print(train_idx)
mnist_train.train_data = mnist_train.train_data[train_idx]
mnist_train.train_labels = mnist_train.train_labels[train_idx]

test_idx = mnist_test.test_labels <= 1
mnist_test.test_data = mnist_test.test_data[test_idx]
mnist_test.test_labels = mnist_test.test_labels[test_idx]

train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)
self.std = torch.Tensor(std)
def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

# values are standard normalization for ImageNet images, 
# from https://github.com/pytorch/examples/blob/master/imagenet/main.py
norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# load pre-trained ResNet50, and put into evaluation mode (necessary to e.g. turn off batchnorm)
model = resnet50(pretrained=True)
model.eval();

# Predict with the model

In [ ]:
# form predictions
pred = model(norm(pig_tensor))
print(pred.shape)

# get the argmax label and its name
import json

with open("imagenet_class_index.json") as f:
    imagenet_classes = {int(i):x[1] for i,x in json.load(f).items()}
print(imagenet_classes[pred.max(dim=1)[1].item()])

## Loss and confidence in prediction

In [ ]:
# 341 is the class index corresponding to "hog"
pred_loss = nn.CrossEntropyLoss()(model(norm(pig_tensor)),torch.m([341])).item()
print("Loss = ", pred_loss)
print("Confidence in prediction = ", np.exp(-pred_loss))

# Let there be noise

In [ ]:
import torch.optim as optim
# The noise radius
epsilon = 2./255

delta = torch.zeros_like(pig_tensor, requires_grad=True)
opt = optim.SGD([delta], lr=1e-1)

for t in range(30):
    pred = model(norm(pig_tensor + delta))
    loss = -nn.CrossEntropyLoss()(pred, torch.LongTensor([341]))
    if t % 5 == 0:
        print(t, loss.item())
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    delta.data.clamp_(-epsilon, epsilon)
    
print("True class: ", imagenet_classes[341])
print("True class probability:", nn.Softmax(dim=1)(pred)[0,341].item())

In [ ]:
max_class = pred.max(dim=1)[1].item()
print("Predicted class: ", imagenet_classes[max_class])
print("Confidence in prediction:", nn.Softmax(dim=1)(pred)[0,max_class].item())

## What made this change?

In [ ]:
plt.imshow((pig_tensor + delta)[0].detach().numpy().transpose(1,2,0)) #Perturbed

In [ ]:
plt.imshow(pig_tensor[0].detach().numpy().transpose(1,2,0)) #Original

In [ ]:
plt.imshow((50*delta+0.5)[0].detach().numpy().transpose(1,2,0)) #Added noise

What you have seen right now is a naive adversarial attack! Let's check the math behind.

# Let's look into Linear models

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

mnist_train = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("./data", train=False, download=True, transform=transforms.ToTensor())

train_idx = mnist_train.train_labels <= 1
mnist_train.train_data = mnist_train.train_data[train_idx]
mnist_train.train_labels = mnist_train.train_labels[train_idx]

test_idx = mnist_test.test_labels <= 1
mnist_test.test_data = mnist_test.test_data[test_idx]
mnist_test.test_labels = mnist_test.test_labels[test_idx]

train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)